# Preamble

In [ ]:
# !pip install --upgrade geoplot
# !conda install descartes -y

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import re
import numpy as np
import pandas as pd
import itertools
import configparser
import json
import requests
config=configparser.ConfigParser()

from ast import literal_eval

from collections import Counter
from floweaver import *
from scipy import stats
from itertools import count


# from contraction import *

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords', quiet=True)
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

import gensim
import gensim.corpora as corpora
from gensim.models.phrases import Phrases, Phraser

import geopandas as gpd
# import geoplot
import pgeocode
nomi = pgeocode.Nominatim('gb')
import shapely

import string
punct = set(string.punctuation)

#
 

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import sentencepiece as spm
USE_LITE = "https://tfhub.dev/google/universal-sentence-encoder-lite/2"
module = hub.Module(USE_LITE)
def process_to_IDs_in_sparse_format(sp, documents):
    """Process documents with the SentencePiece processor. The results have a format
    similar to tf.SparseTensor (values, indices, dense_shape)."""
    ids = [sp.EncodeAsIds(x) for x in documents]
    max_len = max(len(x) for x in ids)
    dense_shape = (len(ids), max_len)
    values = [item for sublist in ids for item in sublist]
    indices = [[row, col] for row in range(len(ids)) for col in range(len(ids[row]))]
    return (values, indices, dense_shape)
def docs2vectors(documents, module):
    """Find the vector representation of a collection of documents using Google's
    Universal Sentence Encoder (lite) model.
    Args:
        documents (:obj:`list` of :obj:`str`): List of raw text documents.
    Returns:
        doc_embeddings (:obj:`numpy.array` of :obj:`numpy.array` of :obj:`float`): Vector representation of documents.
    """
    input_placeholder = tf.compat.v1.sparse_placeholder(tf.int64, shape=[None, None])
    encodings = module(
        inputs=dict(
            values=input_placeholder.values,
            indices=input_placeholder.indices,
            dense_shape=input_placeholder.dense_shape))
    with tf.compat.v1.Session() as sess:
        spm_path = sess.run(module(signature="spm_path"))
        sp = spm.SentencePieceProcessor()
        sp.Load(spm_path)
        # Preprocess documents
        values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, documents)
        sess.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])
        doc_embeddings = sess.run(
                                encodings,
                                feed_dict={input_placeholder.values: values,
                                           input_placeholder.indices: indices,
                                           input_placeholder.dense_shape: dense_shape})
    return doc_embeddings
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
!pip install "tensorflow==1.7"

In [ ]:
!pip install tensorflow-hub

# Main

## Look at Data

In [ ]:
df_over_5k = pd.read_csv('../../data/interim/df_grants_5k_and_over_df.csv', index_col=[0])
# df_under_5k = pd.read_csv('data/df_grants_under_5k_df.csv')

df_over_5k_media = pd.read_csv('../../data/interim/grants_5k_and_over_media_all.csv', index_col=[0])
# df_under_5k_media = pd.read_csv('data/grants_under_5k_media_all.csv')

df_over_5k_gb = pd.read_csv('../../data/interim/grants_5k_and_over_england_filtered_05-12-19.csv', index_col=[0])
df_over_5k_gb = df_over_5k_gb[df_over_5k_gb.columns[1:]]
# df_under_5k_eng = pd.read_csv('data/grants_under_5k_england_filtered_05-12-19.csv')

df_over_5k_gb_media = pd.read_csv('../../data/interim/grants_5k_and_over_england_media_filtered_05-12-19.csv', index_col=[0])
# df_under_5k_eng_media = pd.read_csv('data/grants_under_5k_england_media_filtered_05-12-19.csv')


In [ ]:
print(len(df_over_5k))
print(len(df_over_5k_media))

print(len(df_over_5k_gb))
print(len(df_over_5k_gb_media))

*Number of Unique Values per field*

In [ ]:
# df_over_5k[col].is_unique

In [ ]:
for col in df_over_5k.columns:
    print('{}: {}'.format(col,str(df_over_5k[col].is_unique)))

In [ ]:
unique_values = df_over_5k.nunique().sort_values(ascending=False)

In [ ]:
print('Number of Entries: {}'.format(len(df_over_5k)))
print('Number of Grant Keys: {}'.format(unique_values[0]))
unique_values.plot(kind='barh', fontsize=8, figsize=(5,11))
# plt.yticks(ha='right', rotation=45, fontsize=9)
plt.gca().invert_yaxis()
plt.title('Number of Unique values per Column')
plt.ylabel('Column')
plt.xlabel('Frequency')
plt.grid(alpha=0.3)

In [ ]:
df_over_5k[(df_over_5k.duplicated(subset='grant_key', keep=False)==True) & (df_over_5k['grant_key'] == 20464956)]['source_file_type']

In [ ]:
df_over_5k['gm_country'].isnull().value_counts()

---

In [ ]:
#getting rid of dups
df_over_5k.drop_duplicates(subset = 'grant_key',keep = 'last', inplace=True)
df_over_5k_media.drop_duplicates(subset = 'grant_key',keep = 'last', inplace=True)
df_over_5k_gb.drop_duplicates(subset = 'grant_key',keep = 'last', inplace=True)
df_over_5k_gb_media.drop_duplicates(subset = 'grant_key',keep = 'last', inplace=True)

In [ ]:
print(len(df_over_5k))
print(len(df_over_5k_media))
print(len(df_over_5k_gb))
print(len(df_over_5k_gb_media))

---

*Number of Missing Values per Field*

In [ ]:
pd.DataFrame((df_over_5k.isnull()
 .sum().sort_values(ascending=False)/df_over_5k.shape[0])).loc['developing_country']

In [ ]:
(df_over_5k.isnull()
 .sum().sort_values(ascending=False)/df_over_5k.shape[0]).plot(kind='barh', figsize=(5,11))

# plt.xticks(ha='right', rotation=45, fontsize=7)
plt.yticks(fontsize=8)
plt.ylabel('Column')
plt.xlabel('Fraction')
plt.gca().invert_yaxis()
plt.title('Proportion of Missing (Null) Values per Columns')
plt.grid(alpha=0.2)

<span style="font-family: Courier New;">A significant amount of columns have at least 50% of values missing. Additionally, a handful have at least 90% os data missing. Some columns that may be important in the analysis may not be possible to work with because of this. Columns such as:</span>

- recip_country_tran
- gm_country
- duration 

---

*Possible United States Funds*

In [ ]:
df_over_5k_gb['gm_country'].isnull().count()

In [ ]:
df_over_5k_gb[(df_over_5k_gb['gm_country'].isnull())]['gm_city'].value_counts()[:10]

In [ ]:
df_over_5k[(df_over_5k['recip_country'].isnull())]['recip_city'].value_counts()[:10]

In [ ]:
df_over_5k[(df_over_5k['recip_country'].isnull())]['recip_city'].value_counts()[-20:]

<span style="font-family:	Courier New;">Many funds have cities but not country attached. Can safely assume that majority of these as from the United States. A probability that a city may not be in the U.S. but these are low in frequency.</span>

In [ ]:
#assigning U.S./United States
df_over_5k['gm_country'].fillna('US', inplace=True)
df_over_5k['recip_country'].fillna('US', inplace=True)
# df_over_5k_media['gm_country'].fillna('US', inplace=True)
# df_over_5k_media['recip_country'].fillna('US', inplace=True)
df_over_5k_gb['gm_country'].fillna('US', inplace=True)
df_over_5k_gb['recip_country'].fillna('US', inplace=True)
# df_over_5k_england_media['gm_country'].fillna('US', inplace=True)
# df_over_5k_england_media['recip_country'].fillna('US', inplace=True)

df_over_5k['gm_country_tran'].fillna('United States', inplace=True)
df_over_5k['recip_country_tran'].fillna('United States', inplace=True)
# df_over_5k_media['gm_country_tran'].fillna('United States', inplace=True)
# df_over_5k_media['recip_country_tran'].fillna('United States', inplace=True)
df_over_5k_gb['gm_country_tran'].fillna('United States', inplace=True)
df_over_5k_gb['recip_country_tran'].fillna('United States', inplace=True)
# df_over_5k_england_media['gm_country_tran'].fillna('United States', inplace=True)
# df_over_5k_england_media['recip_country_tran'].fillna('United States', inplace=True)

So quick overview
dataframe uk includes all grants sent to the U.K.! So also includes international GMs and  obvs not all U.K. GMs in the uk df since some are sent outside! Hence for `df_over_5k_all_uk_gms`

In [ ]:

df_over_5k_all_gb_gms = df_over_5k[df_over_5k['gm_country'] =='GB']

---

In [ ]:
df_over_5k.columns

<span style="font-family:	Courier New;">
    Around 20,000 unique values in the "Amount" field. The number of grant keys is less than the total amount of entries.
</span>

- **Tend to invest in round numbers?**

- **Installments of investment? (do duplicates on keys and look at amount)**

---

###

In [ ]:
df_under_5k['gm_country'].fillna('US', inplace=True)
df_under_5k['recip_country'].fillna('US', inplace=True)

df_under_5k['gm_country_tran'].fillna('United States', inplace=True)
df_under_5k['recip_country_tran'].fillna('United States', inplace=True)

###

In [ ]:
fnf_keywords = [
    'news', 'public', 'interest',
    'journalism', 'journalist', 'journalists',
    'innovation', 'innovate', 'innovative',
    'engagement', 'engage', 'engaging', 'engaged',
    'media', 'podcast', 'podcasts', 'digital',
    'video', 'videos', 'film', 'films', 'radio',
    'print', 'printing', 'newspapers', 'newspaper',
    'magazine', 'magazines', 'publishing', 'publisher', 'publishers', 'publish',
    'broadcast', 'broadcasting'
]

In [ ]:
eng_list = df_over_5k[(df_over_5k['recip_country'] =='GB')]['recip_city'].value_counts().keys().to_list()
eng_under_list = df_under_5k[(df_under_5k['recip_country'] =='GB')]['recip_city'].value_counts().keys().to_list()


In [ ]:
with open('data/fnf_keywords.txt') as f:
    fnf_cities = list(literal_eval(f.read()))
    

# with open('file.txt') as f:
#     dictionaries = literal_eval(f.read().strip()) 

In [ ]:
df_grants_over_5k_2 = df_over_5k[(df_over_5k['recip_country'] == 'GB') | (df_over_5k['recip_country_tran'] == 'United Kingdom')]
df_grants_over_5k_2['recip_name'] = df_grants_over_5k_2['recip_name'].fillna('')
df_grants_over_5k_2['description'] = df_grants_over_5k_2['description'].fillna('')
df_grants_over_5k_england = df_grants_over_5k_2.loc[(df_grants_over_5k_2['recip_city'].isin(fnf_cities)) | (df_grants_over_5k_2['recip_city'].fillna('') == '')]

df_grants_under_5k_2 = df_under_5k[(df_under_5k['recip_country'] == 'GB') | (df_under_5k['recip_country_tran'] == 'United Kingdom')]
df_grants_under_5k_2['recip_name'] = df_grants_under_5k_2['recip_name'].fillna('')
df_grants_under_5k_2['description'] = df_grants_under_5k_2['description'].fillna('')
df_grants_under_5k_england = df_grants_under_5k_2.loc[(df_grants_under_5k_2['recip_city'].isin(fnf_cities)) | (df_grants_under_5k_2['recip_city'].fillna('') == '')]


df_grants_under_5k_england['descrip_and_name'] = df_grants_under_5k_england['recip_name'] + ' ' + df_grants_under_5k_england['description']
df_grants_over_5k_england['descrip_and_name'] = df_grants_over_5k_england['recip_name'] + ' ' + df_grants_over_5k_england['description']

In [ ]:
df_grants_under_5k_england.to_csv('data/grants_under_5k_england_filtered_05-12-19.csv')

df_grants_over_5k_england.to_csv('data/grants_5k_and_over_england_filtered_05-12-19.csv')


In [ ]:
# tokenise to filter out mmedia
# df_grants_over_5k_all['tokens'] = df_grants_over_5k_all['descrip_and_name'].apply(lambda x: x.lower()).apply(word_tokenize)
# df_grants_under_5k_all['tokens'] = df_grants_under_5k_all['descrip_and_name'].apply(lambda x: x.lower()).apply(word_tokenize)

df_grants_under_5k_england['tokens'] = df_grants_under_5k_england['descrip_and_name'].apply(lambda x: x.lower()).apply(word_tokenize)
df_grants_over_5k_england['tokens'] = df_grants_over_5k_england['descrip_and_name'].apply(lambda x: x.lower()).apply(word_tokenize)

In [ ]:
df_grants_under_5k_england_filtered = df_grants_under_5k_england[df_grants_under_5k_england['tokens'].apply(lambda x: any(t in fnf_keywords for t in x)) == True]

df_grants_over_5k_england_filtered = df_grants_over_5k_england[df_grants_over_5k_england['tokens'].apply(lambda x: any(t in fnf_keywords for t in x)) == True]

In [ ]:
df_grants_over_5k_england_filtered

In [ ]:
len(df_grants_under_5k_england_filtered)

In [ ]:
df_grants_under_5k_england_filtered.to_csv('data/grants_under_5k_england_media_filtered_05-12-19.csv')
df_grants_over_5k_england_filtered.to_csv('data/grants_5k_and_over_england_media_filtered_05-12-19.csv')

In [ ]:
df_grants_over_5k_england_filtered_less = df_grants_over_5k_england_filtered[['grant_key','gm_name', 'gm_country', 'gm_city', 'recip_name',
                                       'recip_country', 'recip_city', 'amount', 'yr_issued', 'duration']]

df_grants_under_5k_england_filtered_less = df_grants_under_5k_england_filtered[['grant_key','gm_name', 'gm_country', 'gm_city', 'recip_name',
                                       'recip_country', 'recip_city', 'amount', 'yr_issued', 'duration']]


In [ ]:
df_grants_under_5k_england_filtered_less.to_csv('data/grants_under_5k_england_media_filtered_lesscols_05-12-19.csv')
df_grants_over_5k_england_filtered_less.to_csv('data/grants_5k_and_over_england_media_filtered_lesscols_05-12-19.csv')

In [ ]:
df_over_5k_england_media

In [ ]:
df_over_5k_england_media = pd.read_csv('data/grants_5k_and_over_england_media_filtered.csv')
df_under_5k_england_media = pd.read_csv('data/grants_under_5k_england_media_filtered.csv')

In [ ]:
df_over = df_over_5k_england_media[['grant_key','gm_name', 'gm_country', 'gm_city', 'recip_name',
                                       'recip_country', 'recip_city', 'amount', 'yr_issued', 'duration']]
common_over = df_grants_over_5k_england_filtered_less.merge(df_over, on = 'grant_key')
df_grants_over_5k_england_filtered_less[(~df_grants_over_5k_england_filtered_less.grant_key.isin(common_over.grant_key))].to_csv('data/grants_over_5k_media_england_05_12_19.csv')

In [ ]:
df_under = df_under_5k_england_media[['grant_key','gm_name', 'gm_country', 'gm_city', 'recip_name',
                                       'recip_country', 'recip_city', 'amount', 'yr_issued', 'duration']]
common_over = df_grants_under_5k_england_filtered_less.merge(df_under, on = 'grant_key')
df_grants_under_5k_england_filtered_less[(~df_grants_under_5k_england_filtered_less.grant_key.isin(common_over.grant_key))]

In [ ]:
len(df_grants_over_5k_england_filtered_less)

In [ ]:
df_over_5k_england_media

---

## Analysis

### The Grants

#### Top grant makers & grant recipients (number of projects)

- GM that have rewarded the most (by number of projects)
- Recips that have received the most (by number of projects)

Global

In [ ]:
figs, ax = plt.subplots(ncols=2, figsize=(12,4.5))
df_over_5k['gm_name'].value_counts()[:20].sort_values(ascending=False).plot(kind='barh', fontsize=9, ax=ax[0], color = '#0022f5')
df_over_5k['recip_name'].value_counts()[:20].sort_values(ascending=False).plot(kind='barh', fontsize=9, ax=ax[1], color = '#0022f5')
# ax[0].grid(alpha=0.1)
# ax[1].grid(alpha=0.1)

ax[0].set_title('Top Grant Makers', fontsize=9)
ax[0].set_xlabel('Number of Grants Awarded')
ax[0].invert_yaxis()

ax[1].set_title('Top Frequent Recipients of Grants', fontsize=9)
ax[1].set_xlabel('Number of Grants Received')
ax[1].invert_yaxis()
plt.tight_layout()

In [ ]:
print('Top 20 make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k['gm_name'].value_counts()[:20].sort_values(ascending=False).sum()/len(df_over_5k))*100,len(df_over_5k)))
print('Top 20 recipients make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k['recip_name'].value_counts()[:20].sort_values(ascending=False).sum()/len(df_over_5k))*100, len(df_over_5k)))

UK only

In [ ]:
####UK
figs, ax = plt.subplots(ncols=2, figsize=(12,4.5))
df_over_5k_all_gb_gms['gm_name'].value_counts()[:20].sort_values(ascending=False).plot(kind='barh', fontsize=9, ax=ax[0], color = '#0022f5')
df_over_5k_gb['recip_name'].value_counts()[:20].sort_values(ascending=False).plot(kind='barh', fontsize=9, ax=ax[1], color = '#0022f5')
# ax[0].grid(alpha=0.1)
# ax[1].grid(alpha=0.1)

ax[0].set_title('Top Grant Makers', fontsize=9)
ax[0].set_xlabel('Number of Grants Awarded')
ax[0].invert_yaxis()

ax[1].set_title('Top Frequent Recipients of Grants', fontsize=9)
ax[1].set_xlabel('Number of Grants Received')
ax[1].invert_yaxis()
plt.tight_layout()

In [ ]:
print('Top 20 UK make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k_all_gb_gms['gm_name'].value_counts()[:20].sort_values(ascending=False).sum()/len(df_over_5k))*100,len(df_over_5k)))
print('Top 20 UK recipients make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k_gb['recip_name'].value_counts()[:20].sort_values(ascending=False).sum()/len(df_over_5k))*100, len(df_over_5k)))

#### Top grant makers and recipients (total amount)

Global

In [ ]:
figs, ax = plt.subplots(ncols=2, figsize=(12,4.5))
df_over_5k.pivot_table(index=df_over_5k[['gm_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20].plot(kind='barh', fontsize=9, ax=ax[0], logx=True, color = '#0022f5')
df_over_5k.pivot_table(index=df_over_5k[['recip_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20].plot(kind='barh', fontsize=9, ax=ax[1], logx=True, color = '#0022f5')
ax[0].grid(alpha=0.3)
ax[1].grid(alpha=0.3)

ax[0].set_title('Top Grant Makers')
ax[0].set_xlabel('Amount of funding awarded over the years (log)')
ax[0].invert_yaxis()

ax[1].set_title('Top Frequent Recipients of Grants')
ax[1].set_xlabel('Amount of funding received over the years (log)')
ax[1].invert_yaxis()
plt.tight_layout()


In [ ]:
total_top_20_gm = (df_over_5k.pivot_table(index=df_over_5k[['gm_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20]).sum()
total_top_20_recip = (df_over_5k.pivot_table(index=df_over_5k[['recip_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20]).sum()

In [ ]:
total_funds_gm = df_over_5k.pivot_table(index=df_over_5k[['gm_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False).sum()
total_funds_recip = df_over_5k.pivot_table(index=df_over_5k[['recip_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False).sum()

In [ ]:
print('Top 20 funders make up {:.2f}% of total funding globally'.format((total_top_20_gm/total_funds_gm)*100))
print('Top 20 recipients recieve {:.2f}% of total funding globally'.format((total_top_20_recip/total_funds_recip)*100))

In [ ]:
# print('Top 20 make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k['gm_name'].value_counts()[:20].sort_values(ascending=False).sum()/len(df_over_5k))*100,len(df_over_5k)))
# print('Top 20 recipients make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k['recip_name'].value_counts()[:20].sort_values(ascending=False).sum()/len(df_over_5k))*100, len(df_over_5k)))

U.K. only

In [ ]:
###uk 
figs, ax = plt.subplots(ncols=2, figsize=(12,4.5))

df_over_5k_all_gb_gms.pivot_table(index=df_over_5k_all_gb_gms[['gm_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20].plot(kind='barh', fontsize=9, ax=ax[0], logx=True, color = '#0022f5')
df_over_5k_gb.pivot_table(index=df_over_5k[['recip_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20].plot(kind='barh', fontsize=9, ax=ax[1], logx=True, color = '#0022f5')
ax[0].grid(alpha=0.3)
ax[1].grid(alpha=0.3)

ax[0].set_title('Top Grant Makers')
ax[0].set_xlabel('Amount of funding awarded over the years (log)')
ax[0].invert_yaxis()

ax[1].set_title('Top Frequent Recipients of Grants')
ax[1].set_xlabel('Amount of funding received over the years (log)')
ax[1].invert_yaxis()
plt.tight_layout()


In [ ]:
total_top_20_gm = (df_over_5k_all_gb_gms.pivot_table(index=df_over_5k_all_gb_gms[['gm_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20]).sum()
total_top_20_recip = (df_over_5k_gb.pivot_table(index=df_over_5k_gb[['recip_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20]).sum()

In [ ]:
total_funds_gm = df_over_5k.pivot_table(index=df_over_5k[['gm_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False).sum()
total_funds_recip = df_over_5k.pivot_table(index=df_over_5k[['recip_name']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False).sum()

In [ ]:
print('Top 20 funders make up {:.2f}% of total funding globally'.format((total_top_20_gm/total_funds_gm)*100))
print('Top 20 recipients recieve {:.2f}% of total funding globally'.format((total_top_20_recip/total_funds_recip)*100))

- **Are some of these "projects" grant installments"?**

- Collection of recipients (Anon Recip.)

- groupby on countries and select some of the countries (top recips/funders)

---

#### Top Funders and Recipients Countries (Global)- Number of Projects

In [ ]:
figs, axs = plt.subplots(ncols=2, figsize=(9,4.5))
df_over_5k['gm_country_tran'].value_counts()[:10].plot(kind='barh', ax=axs[0], logx=True, color = '#0022f5')
df_over_5k['recip_country_tran'].value_counts()[:10].plot(kind='barh', ax=axs[1], logx = True, color = '#0022f5')

plt.setp(axs[0].get_xticklabels(), fontsize=8)
plt.setp(axs[1].get_xticklabels(), fontsize=8)

# axs[0].set_ylim(0, 140000)
# axs[1].set_ylim(0, 140000)

axs[0].get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
axs[1].get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

axs[0].set_xlabel('Number of Projects')
axs[0].set_ylabel('Country', fontsize=10)
axs[0].set_title('Top 10 Countries with the most funders', fontsize=10)
axs[0].grid(alpha=0.1)
axs[0].invert_yaxis()

axs[1].set_xlabel('Number of Projects')
axs[1].set_ylabel('Country', fontsize=10)
axs[1].set_title('Top 10 Countries with the most recipients', fontsize=10)
axs[1].grid(alpha=0.1)
axs[1].invert_yaxis()
plt.tight_layout()
# plt.savefig('figs/top_countries_no_projs.png', dpi=300)

In [ ]:
print('Top 10 make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k['gm_country_tran'].value_counts()[:10].sort_values(ascending=False).sum()/len(df_over_5k))*100,len(df_over_5k)))
print('Top 10 recipients make up {:.2f}% of the project space (out of {} projects)'.format((df_over_5k['recip_country_tran'].value_counts()[:10].sort_values(ascending=False).sum()/len(df_over_5k))*100, len(df_over_5k)))

In [ ]:
#non-log

figs, axs = plt.subplots(ncols=2, figsize=(9,4.5))
df_over_5k['gm_country_tran'].value_counts()[:10].plot(kind='barh', ax=axs[0], color = '#0022f5')
df_over_5k['recip_country_tran'].value_counts()[:10].plot(kind='barh', ax=axs[1], color = '#0022f5')

plt.setp(axs[0].get_xticklabels(), fontsize=8)
plt.setp(axs[1].get_xticklabels(), fontsize=8)

# axs[0].set_ylim(0, 140000)
# axs[1].set_ylim(0, 140000)

# axs[0].get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
# axs[1].get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

axs[0].set_xlabel('Number of Projects')
axs[0].set_ylabel('Country', fontsize=10)
axs[0].set_title('Top 10 Countries with the most funders', fontsize=10)
axs[0].grid(alpha=0.1)
axs[0].invert_yaxis()

axs[1].set_xlabel('Number of Projects')
axs[1].set_ylabel('Country', fontsize=10)
axs[1].set_title('Top 10 Countries with the most recipients', fontsize=10)
axs[1].grid(alpha=0.1)
axs[1].invert_yaxis()
plt.tight_layout()
# plt.savefig('figs/top_countries_no_projs.png', dpi=300)

In [ ]:
# df_over_5k['gm_country_tran'].value_counts()[:10]

#### Top Funders and Recipients Countries (Global)- Amount of Funding

In [ ]:
figs, ax = plt.subplots(ncols=2, figsize=(9,4.5))
df_over_5k.pivot_table(index=df_over_5k[['gm_country_tran']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:10].plot(kind='barh', fontsize=9, ax=ax[0], logx=True, color = 'C2')
df_over_5k.pivot_table(index=df_over_5k[['recip_country_tran']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:10].plot(kind='barh', fontsize=9, ax=ax[1], logx=True, color = 'C2')
ax[0].grid(alpha=0.1)
ax[1].grid(alpha=0.1)

plt.setp(ax[0].get_xticklabels(),fontsize=8)
plt.setp(ax[1].get_xticklabels(), fontsize=8)

ax[0].set_title('Top Grant Makers Countries')
ax[0].set_xlabel('Amount of funding awarded over the years')
ax[0].set_ylabel('Country', fontsize=10)
ax[0].invert_yaxis()

ax[1].set_title('Top Frequent Recipient Countries of Grants')
ax[1].set_xlabel('Amount of funding received over the years')
ax[1].set_ylabel('Country', fontsize=10)
ax[1].invert_yaxis()
plt.tight_layout()
plt.savefig('figs/top_countries_total_amount.png', dpi=300)

In [ ]:
total_top_10_gm = (df_over_5k.pivot_table(index=df_over_5k['gm_country_tran'], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:10]).sum()
total_top_10_recip = (df_over_5k.pivot_table(index=df_over_5k['recip_country_tran'], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20]).sum()

In [ ]:
total_funds_gm = df_over_5k.pivot_table(index=df_over_5k['gm_country_tran'], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False).sum()
total_funds_recip = df_over_5k.pivot_table(index=df_over_5k['recip_country_tran'], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False).sum()

In [ ]:
print('Top 10 funders make up {:.2f}% of total funding globally'.format((total_top_10_gm/total_funds_gm)*100))
print('Top 10 recipients recieve {:.2f}% of total funding globally'.format((total_top_10_recip/total_funds_recip)*100))

<span style="font-family:	Courier New;"> 
    Majority of grant makers and recipients are from the United States (peaking at around 130,000). Many of the top 10 funders and recipients include European countries, with United Kingdom being the with the second highest amount of grant makers and recipients. Funders based in devloping countries fall in the top 10 funders (Ghana, Kenya and Brazil) and recipients (SA, Brazil, Kenya and India).
</span>

- **What type of projects in these type of regions (europe/u.s./developing)?**

- heatmap on grantmaker countries and recipient and sum of amounts? or exclude america 

---

#### Top Funders and Recipients Cities (Global)- Number of Projects

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(9,4))
# plt.figure(figsize=(8,5))
# df_over_5k_england['recip_city'].value_counts()[df_over_5k_england['recip_city'].value_counts()>= 10].plot(kind='barh')

df_over_5k['recip_city'].value_counts()[:20].plot(kind='barh', ax=ax[0])
df_over_5k_england['recip_city'].value_counts()[:20].plot(kind='barh', ax=ax[1])

ax[0].set_xlabel('Number of Projects')
ax[0].set_ylabel('City', fontsize=10)
ax[0].set_title('Top 10 cities with the most number of projects', fontsize=10)
ax[0].grid(alpha=0.3)

ax[1].set_xlabel('Number of Projects')
ax[1].set_ylabel('City', fontsize=10)
ax[1].set_title('Top 10 UK cities with most number of projects', fontsize=10)
ax[1].grid(alpha=0.3)

ax[0].grid(alpha=0.2)
ax[0].invert_yaxis()

ax[1].grid(alpha=0.2)
ax[1].invert_yaxis()

fig.tight_layout()

#### Top Funders and Recipients Cities (Global)- Amount of Funding

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(9,4))
# plt.figure(figsize=(8,5))
# df_over_5k_england['recip_city'].value_counts()[df_over_5k_england['recip_city'].value_counts()>= 10].plot(kind='barh')

df_over_5k.pivot_table(index=df_over_5k[['recip_city']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20].plot(kind='barh', fontsize=9, ax=ax[0], logx=True)
df_over_5k_gb.pivot_table(index=df_over_5k[['recip_city']], columns ='yr_issued', aggfunc=np.sum)['amount'].sum(axis=1).sort_values(ascending=False)[:20].plot(kind='barh', fontsize=9, ax=ax[1], logx=True)

ax[0].set_xlabel('Amount of funding received over the years')
ax[0].set_ylabel('City', fontsize=10)
ax[0].set_title('Top 10 cities with most total amount of funding received', fontsize=10)
ax[0].grid(alpha=0.3)

ax[1].set_xlabel('Amount of funding received over the years')
ax[1].set_ylabel('City', fontsize=10)
ax[1].set_title('Top 10 UK cities with most total amount of funding received', fontsize=10)
ax[1].grid(alpha=0.3)

ax[0].grid(alpha=0.2)
ax[0].invert_yaxis()

ax[1].grid(alpha=0.2)
ax[1].invert_yaxis()

fig.tight_layout()

#### Media Projects over Time*/*UK Media Projects against All Media projects#### 

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,6), sharex=True)

# media_year_counts = df_over_5k_media.groupby('yr_issued')['amount'].count()
all_year_counts = df_over_5k.groupby('yr_issued')['amount'].count()
all_year_counts_eng = df_over_5k_gb.groupby('yr_issued')['amount'].count()

# media_year_counts.plot(marker='o', ax=ax[0,0])
# (media_year_counts/all_year_counts).plot(marker='o', ax=ax[0,1])
all_year_counts.plot(ax=ax[0])#marker='o', 
all_year_counts_eng.plot(color='orange', ax=ax[0]) #marker='o', 
(all_year_counts_eng/all_year_counts).plot(ax=ax[1], color='green') #marker='o', 

ax[0].set_xlabel('Year Issued')
ax[0].set_ylabel('Frequency')
ax[0].set_title('Number of Grants')
ax[0].legend(['Grants (All)','UK Grants'])
ax[0].grid(alpha=0.3)

ax[1].set_xlabel('Year Issued')
ax[1].set_ylabel('Fraction')
ax[1].set_title('Fraction of Grants received by UK grantees');
ax[1].grid(alpha=0.3)

# media_england_year_counts = df_over_5k_england_media.groupby('yr_issued')['amount'].count()
# england_year_counts = df_over_5k_england.groupby('yr_issued')['amount'].count()

# media_england_year_counts.plot(marker='o', ax=ax[1,0], color='green')
# (media_england_year_counts/england_year_counts).plot(marker='o', ax=ax[1,1], color='green')

# ax[1,0].set_xlabel('Year Issued')
# ax[1,0].set_ylabel('Frequency')
# ax[1,0].set_title('UK Media Grants')
# ax[1,0].grid(alpha=0.3)

# ax[1,1].set_xlabel('Year Issued')
# ax[1,1].set_ylabel('Fraction')
# ax[1,1].set_title('Fraction of UK Grants that are Media');
# ax[1,1].grid(alpha=0.3)
ax[0].set_ylim(0,24000)
ax[1].set_ylim(0, 0.05)

plt.tight_layout()

<span style="font-family:	Courier New;">The number of projects increase over time 2015, slight decline up to 2017 but sharp decline afterwards. Maybe because of errors in entering projects, missing data or projects which have yet not started. Number of media projects spike in 2015 but have the lowest fraction in terms of all projects in the same year (increase in projects?). Rise in fraction of media projects from 2017 to 2019 is mostly due to noise since not many grants issued that year.</span>

<span style="font-family:	Courier New;">As expected, the number of UK grants, including media grants, is siginificantly less than the total amount. The number of UK media projects sharply peaked in 2015, however did slightly fall in amount as it approached 2015. From here onwards, there was a sharp drop in media grant numbers but did stagnate from 2016-2017. The fraction of media projects across fall in the 50%-60% range for most years. This is true even when the amount of projects fluctuate.</span>

Dips- lag in reporting for 2018-2020 porjs or real decline? how much by (lag)?

- **Year issued- date of decision or date of receipt (money given)?** 

- **Filtered beforehand? at least 40% of projects are media related**

- **Yearly installments on the same project?**

### Variable: Funding Amount

#### Descriptive Stats over time

##### Funding Distribution (Full Media Dataset & UK)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,3))

log_amount = np.log10(df_over_5k['amount'])
log_amount.plot(kind='hist', bins=50, ax=ax[0])
ax[0].set_xlabel('Funding Amount USD (log10)')
ax[0].axvline(log_amount.mean(), linestyle='--', color='black', label='mean: $10^{{{}}}$'.format(log_amount.mean()))
ax[0].axvline(log_amount.quantile(0.5), linestyle='--', color='orange', label='median')
ax[0].legend()
ax[0].set_title('Funding Amounts per Project- Global')
ax[0].set_xlabel('Funding Amount (log10)')
ax[0].set_ylabel('Number of Projects', fontsize=10)
ax[0].set_xticks(np.arange(3.5, 8.5, 0.5))

log_amount = np.log10(df_over_5k_gb['amount'])
log_amount.plot(kind='hist', bins=50, ax=ax[1])
ax[1].set_xlabel('Funding Amount USD (log10)')
ax[1].axvline(log_amount.mean(), linestyle='--', color='black', label='mean: $10^{{{}}}$'.format(log_amount.mean()))
ax[1].axvline(log_amount.quantile(0.5), linestyle='--', color='orange', label='median')
ax[1].legend()
ax[1].set_title('Funding Amounts per Project- UK')
ax[1].set_xlabel('Funding Amount (log10)')
ax[1].set_ylabel('Number of Projects', fontsize=10)
ax[1].set_xticks(np.arange(3.5, 8.5, 0.5))

print('Mean Amount: ${:.2f}'.format(df_over_5k_media['amount'].mean()))
print('Median Amount: ${:.2f}'.format(df_over_5k_media['amount'].quantile(0.5)))
plt.tight_layout()
plt.savefig('../../reports/figures/distrib_total_amount_global_uk.png', dpi=300)

<span style="font-family:	Courier New;">The distribution of the amount seems to be right-tailed in the lower end of the funding issued (As seen in the log plot). Evident in the median funding falling below the mean. Downward slope of projects tend to decrease as the funding amount increases. Spikes seem to happen around round numbers (10K, 100K, etc). Heighest number of projects fall below the 10K mark.</span>

---

<span style="font-family: Courier New;">
    Many projects (approx. 600) fall under the \$1 million mark, with a a peak of approximately 525 projects with grants around the \$20K (10^4.3) mark. One project with a \$10 million grant.
    The median is to the left of the which indicates that this is right-tailed distribution (many projects with lower grants). 
    
</span>

Median is higher than the gloabl media- implies the UK projects are getting more than the global average and interesting since most global prjects are from the U.S. Doe sthis indicate most UK recips are awarded 

In [ ]:
# plt.figure(figsize=(8,5))
# # plt.hist(df_over_5k_media['amount'], bins=50, log=True,density='normed', histtype='step',alpha=0.5) # cumulative=True,
# plt.hist(df_over_5k_england_media['amount'], bins=100, log=True, density='normed', histtype='step',alpha=0.5)

---

##### Funding over time (descriptive analysis)

In [ ]:
###ignore 2018 onwards since great drop

In [ ]:
fund_amount_time = df_over_5k.groupby('yr_issued')['amount'].describe()
fund_amount_time_gb = df_over_5k_gb.groupby('yr_issued')['amount'].describe()#[['25%','50%','mean','75%']]

fund_amount_time_gb.loc[:2017]

In [ ]:
fund_amount_time

In [ ]:
# fund_amount_time.loc[:2018][['25%','50%','mean','75%']].plot(marker='o', cmap='tab20',grid=True)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12,4))#

fund_amount_time.loc[:2017][['25%','50%','mean','75%']].plot(cmap='tab20',ax=ax[0])
fund_amount_time_gb.loc[:2017][['25%','50%','mean','75%']].plot(cmap='tab20',ax=ax[1])
# fund_amount_time_eng_media[['25%','50%','mean','75%']].plot(marker='o', cmap='viridis', ax=ax[1], grid=True)

ax[0].set_ylim(0, 400000)
ax[1].set_ylim(0, 400000)

# ax[0].get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
# ax[1].get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
ax[0].set_xlabel('Year Issued')
ax[0].set_ylabel('Value (USD)')
ax[0].set_title('Total Amount of Funding Received Gloablly')

ax[1].set_xlabel('Year Issued')
ax[1].set_ylabel('Value (USD)')
ax[1].set_title('Total Amount of Funding Received in the UK')

# ax[1].set_xlabel('Year Issued')
# ax[1].set_ylabel('Value (USD)')
# ax[1].set_title('Funds issued to Media Grants in the UK')

print('Maximum total sum amount- All UK projects:')
plt.tight_layout()
plt.savefig('../../reports/figures/total_amount_recieved_years_stats_global_uk.png', dpi=300)

<span style="font-family: Courier New;">Stopped at 2018. The mean both cases tend to be higher than the quantiles. This must be dragged by the maximum values (bias). Median a better view. Spike of aggregated funding in 2017 for all projects in england, spike in 2013 for english media projects.
Median and lower quartile are simialr in both cases.</span>

- Why dip in global and stay stagnant? 
- less consist in big grants year to year in UK (the volatility in the mean reflects this)


##### Median Funding over Time per Country (U.K. v.s. Rest of Top 5)

In [ ]:
df_over_5k['recip_country'].value_counts()

In [ ]:
fund_amount_time_us = df_over_5k[df_over_5k['recip_country']=='US'].groupby('yr_issued')['amount'].median()
fund_amount_time_gb = df_over_5k_gb.groupby('yr_issued')['amount'].median()
fund_amount_time_za = df_over_5k[df_over_5k['recip_country']=='ZA'].groupby('yr_issued')['amount'].median()
fund_amount_time_nl = df_over_5k[df_over_5k['recip_country']=='NL'].groupby('yr_issued')['amount'].median()
fund_amount_time_in = df_over_5k[df_over_5k['recip_country']=='IN'].groupby('yr_issued')['amount'].median()

In [ ]:
time_df_top_5 = pd.DataFrame({'US': fund_amount_time_us, 'GB': fund_amount_time_gb,
             'ZA': fund_amount_time_za, 'NL': fund_amount_time_nl,
             'IN': fund_amount_time_in}).loc[:2017]
time_df_top_5.reset_index(inplace=True)

In [ ]:
time_df_top_5

In [ ]:
# ax=plt.plot( 'yr_issued', 'US', data=time_df_top_5, marker='', color='skyblue', linewidth=2, alpha=0.6)
# ax=plt.plot( 'yr_issued', 'GB', data=time_df_top_5, marker='', color='crimson', linewidth=4)
# ax=plt.plot( 'yr_issued', 'ZA', data=time_df_top_5, marker='', color='olivedrab', linewidth=2, label="ZA", alpha=0.6)
# ax=plt.plot( 'yr_issued', 'NL', data=time_df_top_5, marker='', color='blue', linewidth=2, label="NL", alpha=0.6)
# ax=plt.plot( 'yr_issued', 'IN', data=time_df_top_5, marker='', color='darkorange', linewidth=2, label="IN", alpha=0.6)
# plt
ax= time_df_top_5.plot(x='yr_issued', y = ['US', 'GB', 'ZA', 'NL', 'IN'], 
                       legend=False, figsize=(8,7))
# plt.yscale('log')

for line, name in zip(ax.lines, time_df_top_5[['US', 'GB', 'ZA', 'NL', 'IN']].columns):
    y = line.get_ydata()[-1]
    ax.annotate(name, xy=(1,y), xytext=(6,0.3), color=line.get_color(), 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=14, va="center")
plt.title('Total Median Funding Received Over Time')
plt.ylabel('Median Amount', fontsize=12)
plt.xlabel('Year', fontsize=12)
# plt.legend()
plt.savefig('../../reports/figures/distrib_total_amount_median_top5_and_uk.png', dpi=300)

---

#### Global Funding Distribution (GM to Recipient)

In [ ]:
# df_over_5k_media['gm_country'].value_counts(dropna=False)
plt.figure(figsize=(20,20))

agg_piv = df_over_5k.pivot_table(index=df_over_5k_media['gm_country'], columns ='recip_country', aggfunc=np.sum)['amount']
sns.heatmap(np.log10(agg_piv), square=True, cmap="viridis")

In [ ]:
# agg_piv

In [ ]:
t = df_over_5k.pivot_table(index=df_over_5k[['gm_country','recip_country']], columns ='yr_issued', aggfunc=np.sum)['amount']
t_2 = t.loc[~(t.index.get_level_values('gm_country') == 'US') |~(t.index.get_level_values('recip_country') == 'US')]

WHAT TO DO NEXT:
Drop all the NAN Values- dont't fill with 0 since this makes the network graph too crowded

In [ ]:
t_uk = t.loc[(t.index.get_level_values('gm_country') == 'GB') | (t.index.get_level_values('recip_country') == 'GB')]

In [ ]:
# t_2=t.fillna(0)
# t_uk = t_uk.fillna(0)

In [ ]:
t_2.reset_index(inplace=True)
t_uk.reset_index(inplace=True)

In [ ]:
t_uk

In [ ]:
# t_dict_df = pd.DataFrame(
#     {
#         'source': t_['gm_country'] + '_source',
#         'target': t_['recip_country']+ '_target',
#         'value': t_[2015]
#     }
# )

# t_uk_dict_df = pd.DataFrame(
#     {
#         'source': t_uk['gm_country'] + '_source',
#         'target': t_uk['recip_country'] + '_target',
#         'value': t_uk[2015]
#     }
# )

In [ ]:
import networkx as nx

In [ ]:
# g = nx.from_pandas_edgelist(t_dict_df, 'source', 'target', edge_attr= 'value')
# g_uk = nx.from_pandas_edgelist(t_uk_dict_df, 'source', 'target', edge_attr= 'value')

In [ ]:
# layout_uk = nx.spring_layout(g_uk,iterations=50)

In [ ]:
# d_uk = dict(g_uk.degree)

In [ ]:
# nx.draw_networkx_nodes(g_uk, 
#                        layout,
#                        nodelist= d_uk.keys(),
#                        node_size=[v*50 for v in d_uk.values()], 
#                        node_color='lightblue')

In [ ]:
# nx.draw_networkx(g_uk,
#                 layout,
#                 nodelist= d_uk.keys(),
#                 node_size=[v*50 for v in d_uk.values()], 
#                 node_color='lightblue')

In [ ]:
# t_[2013]

In [ ]:
plt.figure(figsize=(20,15))
g_ = nx.convert_matrix.from_pandas_edgelist(t_2, source = 'gm_country',
                                       target = 'recip_country',
                                      edge_attr=2014, create_using=nx.DiGraph())

layout_ = nx.random_layout(g_)

layout_ = nx.random_layout(g_)

edges, weights = zip(*nx.get_edge_attributes(g_, 2014).items())
d_ = dict(g_.degree())

edges = nx.draw_networkx_edges(g_, layout_,
                               node_size=900, arrowstyle='->',
                               arrowsize=20, #edge_color='b',
                               edgelist = edges, edge_color='black',
                               edge_cmap=plt.cm.Blues, alpha=0.2,
                              width=3)
nodes = nx.draw_networkx_nodes(g_, layout_,
                               node_size=900,
                               node_color=[d[1] for d in g_.in_degree()],
                               cmap= plt.cm.tab20c,
                               with_labels=True,
                               arrows=True,
                               arrow_size=True,
                               width=2,
                              alpha=0.7)
nodes=nx.draw_networkx_nodes(g_, layout_,
                               node_size=900,
                               node_color=[d[1] for d in g_.in_degree()], 
                               cmap= plt.cm.tab20c,
                               arrows=True,
                               arrow_size=True,
                               width=2,
                            alpha=0.7)
labels=nx.draw_networkx_labels(g_, layout_, font_size=15, font_weight='bold')
plt.colorbar(nodes)



In [ ]:
from networkx.algorithms import community

communities_generator = community.girvan_newman(g_)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)

In [ ]:
sorted(map(sorted, next_level_communities))

In [ ]:
# plt.colormaps()

In [ ]:
# list(nx.selfloop_edges(g_uk_))

# colors = [g_uk_[u][v][2015] for u,v in edges]

In [ ]:
plt.figure(figsize=(20,15))
g_uk = nx.convert_matrix.from_pandas_edgelist(t_uk, source = 'gm_country',
                                       target = 'recip_country',
                                      edge_attr=2015, create_using=nx.DiGraph())

layout_ = nx.random_layout(g_uk)

layout_ = nx.random_layout(g_uk)

edges, weights = zip(*nx.get_edge_attributes(g_uk, 2015).items())
d_ = dict(g_uk.degree())

edges = nx.draw_networkx_edges(g_uk, layout_,
                               node_size=900, arrowstyle='->',
                               arrowsize=20, #edge_color='b',
                               edgelist = edges, edge_color='black',
                               edge_cmap=plt.cm.Blues, alpha=0.2,
                              width=3)
nodes = nx.draw_networkx_nodes(g_uk, layout_,
                               node_size=900,
                               node_color=[d[1] for d in g_uk.out_degree()],
                               cmap= plt.cm.tab20c,
                               with_labels=True,
                               arrows=True,
                               arrow_size=True,
                               width=2,
                              alpha=0.7)
nodes=nx.draw_networkx_nodes(g_uk, layout_,
                               node_size=900,
                               node_color=[d[1] for d in g_uk.out_degree()], 
                               cmap= plt.cm.tab20c,
                               arrows=True,
                               arrow_size=True,
                               width=2,
                            alpha=0.7)
labels=nx.draw_networkx_labels(g_uk, layout_, font_size=15, font_weight='bold')
plt.colorbar(nodes)



In [ ]:
# g_uk.edges()
# nx.get_edge_attributes(g_uk,2015)

import matplotlib.cm as cm
import matplotlib as mpl


m = cm.ScalarMappable(norm=norm_, cmap=plt.cm.Blues)

[norm_(i) for i in nx.get_edge_attributes(g_uk,2015).values()]
# m.to_rgba(nx.get_edge_attributes(g_uk,2015))

In [ ]:
[d[1] for d in g_.in_degree()]

In [ ]:
g_uk_.attr

In [ ]:
d_uk = dict(g_uk_.degree())

In [ ]:
layout_uk = nx.spring_layout(g_uk,iterations=50)

In [ ]:
# plt.figure(figsize=(12,10))
# nx.draw_networkx(g_uk,
#                 layout_uk,
#                 nodelist= d_uk.keys(),
#                 node_size=20, 
#                 node_color='lightblue', 
#                 edge_color = 'gray',
#                 font_size = 8,
#                 arrows=True)

---

In [ ]:
df_over_5k[df_over_5k['recip_country'] == 'GB']['recip_city'].value_counts()

---

In [ ]:
# fig, ax = plt.subplots(ncols=2,figsize = (10,5))
# df_over_5k_media.groupby('yr_issued')['amount'].mean().plot(kind='bar', ax=ax[0])
# ax1 = fig.add_subplot(111)
# df_over_5k[df_over_5k['is_media']==False].groupby('yr_issued')['amount'].mean().plot(kind='bar', ax=ax1)


In [ ]:
# plt.scatter(df_over_5k[(np.abs(stats.zscore(df_over_5k['amount'])) < 70)].groupby('yr_issued')['amount'].index,
#             df_over_5k[(np.abs(stats.zscore(df_over_5k['amount'])) < 70)].groupby('yr_issued')['amount'].values)

In [ ]:
df_over_5k[(np.abs(stats.zscore(df_over_5k['amount'])) < 70)].groupby('yr_issued')['amount'].count()

In [ ]:
np.abs(stats.zscore(df_over_5k['amount'])>).tolist()#.sort(reverse=True)

In [ ]:
# df_over_5k['amount']

In [ ]:
# df_over_5k_england_media.groupby('yr_issued')['amount'].
# df_over_5k_england_media['amount'].plot(kind='scatter')

In [ ]:
df_over_5k['over_100char'] = df_over_5k['description'].str.len() > 100

In [ ]:
df_over_5k['dup']=df_over_5k['description'].apply(lambda x: x.lower() if type(x) == str else x).duplicated()

In [ ]:
# df_over_5k['description'][]
df_media = df_over_5k[(df_over_5k['over_100char'] == True) & (df_over_5k['dup'] ==False)]

In [ ]:
df_media['description'][:10][1]

#### Funding Intervals?

In [ ]:
df_over_5k

In [ ]:
dup = df_over_5k[df_over_5k['grant_key'].duplicated(False)]#.loc[[1,2]]#.T
dup

In [ ]:
dup['grant_key'].value_counts()

In [ ]:
dup[dup['grant_key'] == 20464956]#['yr_issued']

In [ ]:
for col in df_over_5k.columns:
#     print(col)
    print(dup[dup['grant_key'] == 20464956][col].duplicated(False))

In [ ]:
# dup.loc[102408]
dup[dup['grant_key'] == 20464956]['source_file_type']

In [ ]:
dup[~dup['fund_type'].isnull()]['fund_type']

In [ ]:
import itertools

In [ ]:
col_pairs = list(itertools.combinations(df_over_5k.columns,2))

In [ ]:
for col in df_over_5k.columns:
#     print(col)
    print(dup[col].duplicated(False))

In [ ]:
dup['source_file_type']

<span style="font-family: Courier New;">Seems like there are a few duplicates in this dataset which are not picked since there may be a slight discrepency in one of the duplicates. Approx. 1689 dups.</span>

### Variable: Geomaps

In [ ]:
poly_gdf = gpd.read_file('../../data/raw/NUTS_RG_01M_2016_4326_LEVL_1.shp/NUTS_RG_01M_2016_4326_LEVL_1.shp')
# points_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
# points_in_poly = gpd.sjoin(points_gdf, poly_gdf, op='within')

In [ ]:
poly_uk = poly_gdf[poly_gdf['CNTR_CODE']=='UK']

# poly_uk.rename(columns={'geometry':'geo_poly'}, inplace=True)

In [ ]:
df_over_5k_gb['city_country'] = df_over_5k_gb['recip_city'] + ', ' + df_over_5k_gb['recip_country_tran']

---

using google geocoding to get lat lon of cities in U.K.

In [ ]:
config.read('../../.env')
api_key = config['geocode']['api_key']

In [ ]:
df_over_5k_gb['city_country'].unique()

In [ ]:
# https://maps.googleapis.com/maps/api/geocode/

In [ ]:
def geocode(address):
    params={'address': address, 'key': api_key}
    headers = {}
    url='https://maps.googleapis.com/maps/api/geocode/json'
    r = requests.get(url=url, params=params, headers=headers)
    if r.status_code == 200:
        
    
        return r.json()['results'][0]['geometry']['location']
    else:
        return None

In [ ]:
%%time
lat_lon_map = {}
for i in df_over_5k_gb['city_country'].unique():
    lat_lon_map[i] = geocode(i)

In [ ]:
#unpacking the dicts so can easily map
lat_dict ={}
for i,k in lat_lon_map.items():
    lat_dict[i] = k['lat']
    

In [ ]:
lon_dict ={}
for i,k in lat_lon_map.items():
    lon_dict[i] = k['lng']

In [ ]:
with open('../../data/interim/lat_dict.json','w') as f:
    json.dump(lat_dict,f)
    
with open('../../data/interim/lon_dict.json','w') as f:
    json.dump(lon_dict,f)    

---

In [ ]:
with open('../../data/interim/lat_dict.json','r') as f:
    lat_dict = json.load(f)
    
with open('../../data/interim/lon_dict.json','r') as f:
    lon_dict = json.load(f)    

In [ ]:
df_over_5k_gb['lat'] = df_over_5k_gb['city_country'].map(lat_dict)
df_over_5k_gb['lon'] = df_over_5k_gb['city_country'].map(lon_dict)

In [ ]:
#amount of projects with location info
df_over_5k_gb['lon'].notnull().sum()

In [ ]:
points_gdf = gpd.GeoDataFrame(df_over_5k_gb, geometry=gpd.points_from_xy(df_over_5k_gb['lon'], df_over_5k_gb['lat']))
points_in_poly = gpd.sjoin(points_gdf, poly_uk, op='within', how='right')

In [ ]:
x = poly_uk[['NUTS_NAME', 'geometry']]
x = x.sort_values(by='NUTS_NAME')

###### mean & median amount

In [ ]:
#mean
x['amount'] = points_in_poly[['NUTS_NAME', 'amount']].sort_values(by='NUTS_NAME').groupby('NUTS_NAME').mean()['amount'].values

In [ ]:
x.plot(column='amount', figsize = (8,10), legend=True).axis('off')
# %%time
# point_poly_amount_total = points_in_poly[['NUTS_NAME', 'geometry', 'amount']].dissolve(by='NUTS_NAME', aggfunc='sum')

In [ ]:
#median
x['amount'] = points_in_poly[['NUTS_NAME', 'amount']].sort_values(by='NUTS_NAME').groupby('NUTS_NAME').median()['amount'].values

In [ ]:
x.plot(column='amount', figsize = (8,10), legend=True).axis('off')

###### number of projects

In [ ]:
x['count'] =points_in_poly[['NUTS_NAME', 'amount']].sort_values(by='NUTS_NAME').groupby('NUTS_NAME').count()['amount'].values

In [ ]:
x.plot(column='count', figsize = (8,10), legend=True).axis('off')

### Variable: Description

In [ ]:
#get descrips thta are at least over 100 chars and are unique

In [ ]:
# from nesta.packages.nlp_utils.ngrammer import Ngrammer
# ngrammer = Ngrammer("/mnt/c/Users/aotubusen/Documents/innovation-mapping.config")

TF-IDF doc-feature matrix creation

In [ ]:
# x = 'hey how are you running!'

In [ ]:
stop_words = set(stopwords.words('english') +
                 ['grant', 'general', 'project', 'support','new', 'give',
                  'initiative', 'city', 'community', 'use', 'provide', 'information',
                 'program', 'training', 'online', 'development', 'develop',
                 'help', 'funding', 'fund']
)

In [ ]:
def tokenise_and_punc_remove(sentence):
    p = ' '.join([contractions.get(i, i) for i in sentence.lower().split()])
    
    clean_tokens = [w for w in word_tokenize(p.lower()) if (w not in stop_words)]
    clean_tokens_ = [re.sub(r'[^\w\s]','',w) for w in clean_tokens]
    clean_tokens__ = [w for w in clean_tokens_ if len(w) > 1]
    return clean_tokens__

In [ ]:
def lem_tokenizer(text):
    results = []
    for i in text:
        lem = WordNetLemmatizer().lemmatize(i, pos='v')
        results.append(lem)
    return results

In [ ]:
tokenise_and_punc_remove("Adeola's hi.k bu")

Decided to stick to bigrams only since unigrams are simialr across the board and the trigrams that appear in top 20 features are not thta informative (see below).

In [ ]:
descrips_df = df_over_5k.loc[df_over_5k['description'].str.len() >= 100]
descrips_df_gb = df_over_5k_gb.loc[df_over_5k_gb['description'].str.len() >= 100]#['description']

In [ ]:
descrips_df_gb

Generate bigrams

In [ ]:
descrips_df_unique = descrips_df.drop_duplicates(subset='description', keep='last')
descrips_df_gb_unique = descrips_df_gb.drop_duplicates(subset='description', keep='last')

In [ ]:
# descrips_df_unique

In [ ]:
descrips_df_unique.reset_index(inplace=True, drop=True)
descrips_df_gb_unique.reset_index(inplace=True, drop=True)

In [ ]:
len(descrips_df_gb_unique)

In [ ]:
%%time
descrips_df_unique['tokens'] = descrips_df_unique['description'].apply(lambda x: lem_tokenizer(tokenise_and_punc_remove(x)))

descrips_df_gb_unique['tokens'] = descrips_df_gb_unique['description'].apply(lambda x: lem_tokenizer(tokenise_and_punc_remove(x)))

In [ ]:
descrips_df_unique.to_csv('../../data/processed/descrips_df_unique_sent_df.csv', index=False)
descrips_df_gb_unique.to_csv('../../data/processed/descrips_df_eng_unique_sent_df.csv', index=False)

In [ ]:
#read csv
descrips_df_unique_sent = pd.read_csv('../../data/processed/descrips_df_unique_sent_df.csv',converters={'tokens':literal_eval})
descrips_df_gb_unique_sent = pd.read_csv('../../data/processed/descrips_df_eng_unique_sent_df.csv',converters={'tokens':literal_eval})

In [ ]:
descrips_df_eng_unique_sent

In [ ]:
phrases = Phrases(descrips_df_unique_sent['tokens'], min_count=1, threshold=1)
bigram = Phraser(phrases)

In [ ]:
# descrips_df_unique_bigrams = descrips_df_unique_sent.apply(lambda x: [i for i in bigram[x] if '_' in i])
# descrips_df_eng_unique_bigrams = descrips_df_eng_unique_sent.apply(lambda x: [i for i in bigram[x] if '_' in i])

descrips_df_unique_ngrams = descrips_df_unique_sent['tokens'].apply(lambda x: bigram[x])
descrips_df_gb_unique_ngrams = descrips_df_gb_unique_sent['tokens'].apply(lambda x: bigram[x])


In [ ]:
# tokens = descrips_df_unique.apply(lambda x: list(itertools.chain.from_iterable(ngrammer.process_document(x[0]))), axis=1)
# tokens_eng = descrips_df_eng_unique.apply(lambda x: list(itertools.chain.from_iterable(ngrammer.process_document(x[0]))), axis=1)

Apply TF-IDF to generate scores matrix

In [ ]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer,
                        stop_words=None,
                        lowercase=False,
                        min_df=2,
                       ngram_range=(2,2)) 
# tfidf_eng = TfidfVectorizer(tokenizer=identity_tokenizer,
#                         stop_words=None,
#                         lowercase=False,
#                         min_df=2) 

- Doing `ngram_range` even AFTER bigram phraser because e.g. if terms `provide children` and `provide adults` are important contextually but unigram `provide` only made it past the significance test and not `adult` and `child` in the gensim phraser, we lose context of who and what is being provided. e.g. providing children may give context of education whilst providing adults maybe be resources for work

In [ ]:
docs_features = tfidf.fit_transform(descrips_df_gb_unique_ngrams)
# docs_features_eng = tfidf.fit_transform(descrips_df_eng_unique_bigrams)

In [ ]:
docs_features
# docs_features_eng

In [ ]:
doc_feat_df = pd.DataFrame(docs_features.todense())
# doc_feat_eng_df = pd.DataFrame(docs_features_eng.todense())

In [ ]:
# rerun to perserve ID from original data 
# update: already sorted out the perservation- just rerun and check indexes

LDA for feature creation for clustering

In [ ]:
from gensim.sklearn_api.ldamodel import LdaTransformer

In [ ]:
dictionary = gensim.corpora.dictionary.Dictionary(descrips_df_gb_unique_ngrams)
corpus = [dictionary.doc2bow(text) for text in descrips_df_gb_unique_ngrams]

In [ ]:
%%time
lda = LdaTransformer(num_topics=12, id2word=dictionary, minimum_probability=0)
lda_doc_vecs = lda.fit_transform(corpus)
#100 topics 100-104 cluster = elbow
#30topics 29-31 clusters = elbow

# transfor

In [ ]:
lda_doc_vecs[0]

---
Searching for optimal number of clusters

###### KMeans

In [ ]:
%%time
distortions = []
K = range(1,25,2)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(lda_doc_vecs)
    distortions.append(kmeanModel.inertia_)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()
## seems like the number of topics coincide with the number of clusters (case for number under 100)

In [ ]:
# defin no. of clusters
# siluette score
# explained method


---

In [ ]:
%%time
cluster_results = KMeans(n_clusters=11).fit_predict(lda_doc_vecs)

In [ ]:
from collections import Counter
Counter(cluster_results)#

In [ ]:
cluster_0

In [ ]:
%%time
##### get top 10 tfidf values for clusters
cluster_0 = list(doc_feat_df.loc[np.where(cluster_results == 0)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_1 = list(doc_feat_df.loc[np.where(cluster_results == 1)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_2 = list(doc_feat_df.loc[np.where(cluster_results == 2)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_3 = list(doc_feat_df.loc[np.where(cluster_results == 3)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_4 = list(doc_feat_df.loc[np.where(cluster_results == 4)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_5 = list(doc_feat_df.loc[np.where(cluster_results == 5)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_6 = list(doc_feat_df.loc[np.where(cluster_results == 6)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_7 = list(doc_feat_df.loc[np.where(cluster_results == 7)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_8 = list(doc_feat_df.loc[np.where(cluster_results == 8)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_9 = list(doc_feat_df.loc[np.where(cluster_results == 9)[0]].sum().sort_values(ascending=False)[:10].index)
cluster_10 = list(doc_feat_df.loc[np.where(cluster_results == 10)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_11 = list(doc_feat_df.loc[np.where(cluster_results == 11)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_12 = list(doc_feat_df.loc[np.where(cluster_results == 12)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_13 = list(doc_feat_df.loc[np.where(cluster_results == 13)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_14 = list(doc_feat_df.loc[np.where(cluster_results == 14)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_15 = list(doc_feat_df.loc[np.where(cluster_results == 15)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_16 = list(doc_feat_df.loc[np.where(cluster_results == 16)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_17 = list(doc_feat_df.loc[np.where(cluster_results == 17)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_18 = list(doc_feat_df.loc[np.where(cluster_results == 18)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_19 = list(doc_feat_df.loc[np.where(cluster_results == 19)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_20 = list(doc_feat_df.loc[np.where(cluster_results == 20)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_21 = list(doc_feat_df.loc[np.where(cluster_results == 21)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_22 = list(doc_feat_df.loc[np.where(cluster_results == 22)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_23 = list(doc_feat_df.loc[np.where(cluster_results == 23)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_24 = list(doc_feat_df.loc[np.where(cluster_results == 24)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_25 = list(doc_feat_df.loc[np.where(cluster_results == 25)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_26 = list(doc_feat_df.loc[np.where(cluster_results == 26)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_27 = list(doc_feat_df.loc[np.where(cluster_results == 27)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_28 = list(doc_feat_df.loc[np.where(cluster_results == 28)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_29 = list(doc_feat_df.loc[np.where(cluster_results == 29)[0]].sum().sort_values(ascending=False)[:10].index)
# cluster_30 = list(doc_feat_df.loc[np.where(cluster_results == 30)[0]].sum().sort_values(ascending=False)[:10].index)

# cluster_9 = list(doc_feat_df.loc[np.where(cluster_results == 9)[0]].sum().sort_values(ascending=False)[:10].index)

In [ ]:
print([tfidf.get_feature_names()[n] for n in cluster_0])
print([tfidf.get_feature_names()[n] for n in cluster_1])
print([tfidf.get_feature_names()[n] for n in cluster_2])
print([tfidf.get_feature_names()[n] for n in cluster_3])
print([tfidf.get_feature_names()[n] for n in cluster_4])
print([tfidf.get_feature_names()[n] for n in cluster_5])
print([tfidf.get_feature_names()[n] for n in cluster_6])
print([tfidf.get_feature_names()[n] for n in cluster_7])
print([tfidf.get_feature_names()[n] for n in cluster_8])
print([tfidf.get_feature_names()[n] for n in cluster_9])
print([tfidf.get_feature_names()[n] for n in cluster_10])
# print([tfidf.get_feature_names()[n] for n in cluster_11])
# print([tfidf.get_feature_names()[n] for n in cluster_12])
# print([tfidf.get_feature_names()[n] for n in cluster_13])
# print([tfidf.get_feature_names()[n] for n in cluster_14])
# print([tfidf.get_feature_names()[n] for n in cluster_15])
# print([tfidf.get_feature_names()[n] for n in cluster_16])
# print([tfidf.get_feature_names()[n] for n in cluster_17])
# print([tfidf.get_feature_names()[n] for n in cluster_18])
# print([tfidf.get_feature_names()[n] for n in cluster_19])
# print([tfidf.get_feature_names()[n] for n in cluster_20])
# print([tfidf.get_feature_names()[n] for n in cluster_21])
# print([tfidf.get_feature_names()[n] for n in cluster_22])
# print([tfidf.get_feature_names()[n] for n in cluster_23])
# print([tfidf.get_feature_names()[n] for n in cluster_24])
# print([tfidf.get_feature_names()[n] for n in cluster_25])
# print([tfidf.get_feature_names()[n] for n in cluster_26])
# print([tfidf.get_feature_names()[n] for n in cluster_27])
# print([tfidf.get_feature_names()[n] for n in cluster_28])
# print([tfidf.get_feature_names()[n] for n in cluster_14])

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

In [ ]:
%%time
# svd = TruncatedSVD(n_components=9)
tsne = TSNE()

# svd_vecs = svd.fit_transform(lda_doc_vecs)
# tsne_vecs = tsne.fit_transform(svd_vecs)

tsne_vecs = tsne.fit_transform(lda_doc_vecs)

In [ ]:
lda_doc_vecs[2]

In [ ]:
lda.gensim_model.print_topics()

In [ ]:
cluster_results

In [ ]:
plt.scatter(tsne_vecs[:, 0], tsne_vecs[:, 1], c=cluster_results, cmap='hsv', alpha=1)

In [ ]:
#tfidf and lda use the same input!
#if tfidf topic words to describe clusters look rubbish, maybe just feed it unigrams 

---

###### DSCAN

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
clustering = DBSCAN(eps=0.8, min_samples=50).fit(lda_doc_vecs)

In [ ]:
Counter(clustering.labels_)

###### NN

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
neigh = NearestNeighbors()
neigh.fit(lda_doc_vecs)

### Draft

In [ ]:
words = []
for j in tokens:
    words.extend(j)
word_count = Counter(words)
top_words= Counter(words).most_common(20)

words_eng = []
for j in tokens_eng:
    words_eng.extend(j)
word_count = Counter(words_eng)
top_england_words= Counter(words_eng).most_common(20)

In [ ]:
len(words)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,6))
ax[0].grid(alpha=0.2)
ax[1].grid(alpha=0.2)

ax[0].set_xlim(0, 10500)
# axs[1].set_ylim(0, 10000)

ax[0].barh(*zip(*top_words), color = 'c')
ax[1].barh(*zip(*top_england_words), color = 'c')

ax[0].set_title('Top 20 words in unique descriptions (Globally)')
ax[1].set_title('Top 20 words in unique descriptions (UK)')

ax[0].invert_yaxis()
ax[1].invert_yaxis()
fig.tight_layout()

In [ ]:
[names[n] for n in list(top_grams[:20])]

In [ ]:
[names[n] for n in list(top_grams[:20])]

---